In [1]:
import requests as http
from io import StringIO
from datetime import datetime

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px

from av_lib import *

In [2]:
api_key = None
with open('api-key.txt') as file:
    api_key = file.read()

In [3]:
symbol = Symbol('AAPL', 'Apple Inc.', 'USA')

In [4]:
url = daily_adjusted_query(api_key, symbol=symbol.tick(), output_size='full')
data = fetch_data(url)

In [5]:
data = StringIO(data)
data = pd.read_csv(data)
data

,timestamp,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient
0,2022-12-09,142.340,145.5700,140.90,142.16,142.160000,76097011,0.0,1.0
1,2022-12-08,142.360,143.5200,141.10,142.65,142.650000,62128338,0.0,1.0
2,2022-12-07,142.190,143.3700,140.00,140.94,140.940000,69721094,0.0,1.0
3,2022-12-06,147.075,147.3000,141.92,142.91,142.910000,64727186,0.0,1.0
4,2022-12-05,147.770,150.9199,145.77,146.63,146.630000,68826442,0.0,1.0
...,...,...,...,...,...,...,...,...,...
5811,1999-11-05,84.620,88.3700,84.00,88.31,0.672034,3721500,0.0,1.0
5812,1999-11-04,82.060,85.3700,80.62,83.62,0.636344,3384700,0.0,1.0
5813,1999-11-03,81.620,83.2500,81.00,81.50,0.620210,2932700,0.0,1.0
5814,1999-11-02,78.000,81.6900,77.31,80.25,0.610698,3564600,0.0,1.0


In [6]:
data['timestamp'] = pd.to_datetime(data['timestamp'])
data.head()

,timestamp,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient
0,2022-12-09,142.340,145.5700,140.90,142.16,142.16,76097011,0.0,1.0
1,2022-12-08,142.360,143.5200,141.10,142.65,142.65,62128338,0.0,1.0
2,2022-12-07,142.190,143.3700,140.00,140.94,140.94,69721094,0.0,1.0
3,2022-12-06,147.075,147.3000,141.92,142.91,142.91,64727186,0.0,1.0
4,2022-12-05,147.770,150.9199,145.77,146.63,146.63,68826442,0.0,1.0


In [7]:
fig = make_candlestick(symbol, data, title=f'Daily Stock Price - {symbol.tick()}')
fig.show()

In [8]:
adj_close = px.line(
    data_frame=data, 
    x='timestamp', 
    y='adjusted_close', 
    title=f'Adjusted Stock Price - {symbol.tick()}',
    labels={
        'timestamp': 'Date',
        'adjusted_close': 'Price'
    },
    log_y=True
)
adj_close.show()

In [9]:
data['timedelta'] = data['timestamp'] - datetime.today()
data.head()

,timestamp,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient,timedelta
0,2022-12-09,142.340,145.5700,140.90,142.16,142.16,76097011,0.0,1.0,-4 days +10:25:44.398777
1,2022-12-08,142.360,143.5200,141.10,142.65,142.65,62128338,0.0,1.0,-5 days +10:25:44.398777
2,2022-12-07,142.190,143.3700,140.00,140.94,140.94,69721094,0.0,1.0,-6 days +10:25:44.398777
3,2022-12-06,147.075,147.3000,141.92,142.91,142.91,64727186,0.0,1.0,-7 days +10:25:44.398777
4,2022-12-05,147.770,150.9199,145.77,146.63,146.63,68826442,0.0,1.0,-8 days +10:25:44.398777


In [10]:
adj_close = px.line(
    data_frame=data, 
    x='timedelta', 
    y='adjusted_close', 
    title=f'Adjusted Stock Price - {symbol.tick()}',
    labels={
        'timedelta': 'Days Before',
        'adjusted_close': 'Price'
    },
    log_y=True
)
adj_close.show()